In [1]:
# Exchange fluxes above the sill
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os

In [7]:
sd = 1 # Sill depth
#/Volumes/Extreme SSD/MITgcm outputs
#/work/oceans/wbao/MITgcm_results/iceplume/fjord_test
#/Users/weiyangbao/.CMVolumes/Caviness-group/iceplume/
file1 = xr.open_dataset('/work/oceans/wbao/MITgcm_results/iceplume/1_BaseCase_nTemp/state_' + str(format(sd,'03d')) + '.nc')
#file2 = xr.open_dataset('/work/oceans/wbao/MITgcm_results/iceplume/shallow_fjord01/icefrntA_' + str(format(sd,'03d')) + '.nc')
T0 = 0 
Tn = len(file1.T) # T0=275~Tn includes N=5 tidal periods (T=12.4 h)
# X=range(1,120), Xp1=range(1,121), Y=range(30,39), Yp1=range(30,40)
#state1 = file1.isel(T=range(T0,Tn)) # Remove the boundary walls and the shelf region
state1 = file1.isel(T=~file1.get_index("T").duplicated())
grid1 = xr.open_dataset('/work/oceans/wbao/MITgcm_results/iceplume/1_BaseCase_nTemp/grid_' + str(format(sd,'03d')) + '.nc')
#file2 = xr.open_dataset('/Users/weiyangbao/Documents/Model_outputs/qf-tide-hourly/state_' + str(format(sd,'03d')) + '.nc')
#state2 = file2.isel(X=range(120), Xp1=range(121), Y=range(30,39), Yp1=range(30,40)) # Remove the boundary walls and the shelf region
#grid2 = xr.open_dataset('/Users/weiyangbao/Documents/Model_outputs/qf-tide-hourly/grid_' + str(format(sd,'03d')) + '.nc')

time1 = state1.T.data
#time2 = state2.T.data / 86400
xs = 10
u1 = state1.U.data[:,:,:,xs]
#u2 = state2.U.data[:,:,:,xs]
#u = state1.U.data[T0,:,5,xs]
dis = state1.X.data
xp1 = state1.Xp1.data

# Area = np.empty([50, 9]) # Cross Y direction
# Area[:10, :] = 100
# Area[10:20, :] = 200
# Area[20:30, :] = 300
# Area[30:40, :] = 800
# Area[40:, :] = 1600

Area = np.empty([45, 10]) # Cross Y direction
Area[:10, :] = 400
Area[10:25, :] = 800
Area[25:, :] = 1200

pt1 = state1.Temp.data[:,:,:,xs]
#pt2 = state2.Temp.data[:,:,:,xs]


s1 = state1.S.data[:,:,:,xs]
#s2 = state2.S.data[:,:,:,xs]



HFacW = grid1.HFacW.data[:,35:45,xs] # vertical fraction of open cell at West face
HFacC = grid1.HFacC
# volume = ds.hFacC * ds.drF * ds.rA
#state1.Xp1.data[72], state1.X.data[71]
#len(time), time
#time1
pt1[0,0,40]

6.043683

In [ ]:
def IFA_shallow(datapath, case_id):
    # Grid areas
    Area = np.empty([90, 10])
    Area[:20,:] = 200
    Area[20:50,:] = 400
    Area[50:,:] = 600
    file0 = xr.open_dataset(datapath+'/icefrntA_' + str(format(case_id,'03d')) + '.nc')
    file = file0.isel(T=~file0.get_index("T").duplicated()) 
    t0 = 0
    #t0 = 89
    tn = len(file.T)
    state = file.isel(Y=range(35,45), T=range(t0,tn))
    MR = state.icefrntA.isel(X=1).data.mean(0) # Melt rate at the icefront
    Qsm = (MR*Area).sum()/(24*3600)
    return Qsm

In [ ]:
path = '/work/oceans/wbao/MITgcm_results/iceplume/shallow_fjord01/'
qsm1 = IFA_shallow(path,3)
qsm1

In [ ]:
fig0, (ax1, ax2, ax3) = plt.subplots(3,1, sharex=True, figsize=(9,9))
ts1 = time1 * 86400
ts2 = time2 * 86400
zi = 40
ax1.plot(ts1,u1[:,zi,40],label="no restart")
ax1.plot(ts2,u2[:,zi,40],lw=.5,label="with restart")
ax1.set_title('$U$', size=15, weight='bold')
ax1.legend(loc = 'best', fontsize=15)
ax1.tick_params(labelsize=15)
ax1.xaxis.set_visible(False)
ax1.set_ylabel('($m/s$)', size=15)


ax2.plot(ts1,pt1[:,zi,40], ts2, pt2[:,zi,40])
ax2.set_title('$Temp$', size=15, weight='bold')
#ax2.legend(loc = 'best', fontsize=15)
ax2.tick_params(labelsize=15)
ax2.xaxis.set_visible(False)
ax2.set_ylabel('($^{\circ}C$)', size=15)



ax3.plot(ts1,s1[:,zi,40], ts2, s2[:,zi,40])
ax3.set_title('$Sal$', size=15, weight='bold')
#ax2.legend(loc = 'best', fontsize=15)
ax3.tick_params(labelsize=15)
ax3.set_ylabel('($psu$)', size=15)

ax3.set_xlabel('Time (s)', size=15)

In [3]:
idx = 130
#dis[idx], xp1[idx], grid1.HFacC.data[:,35,idx]
#grid1.dyF
vol = grid1.HFacC * grid1.drF * grid1.rA # Z Y X
areaC = grid1.HFacC * grid1.drF * grid1.dyF # Z Y X
areaW = grid1.HFacW * grid1.drF * grid1.dyG # Z Y Xp1
temp = state1.Temp.data
s = state1.S.where(HFacC>0)
U = state1.U.where(grid1.HFacW>0)
sW = np.where(HFacC[-1,40,1:] == 0)[-1][0] # Sill west edge index
sE = np.where(HFacC[-1,40,1:] == 0)[-1][-1] # Sill east edge index
sC = np.ceil((sW+sE)/2).astype(int) # Sill crest index (178)
#state1.U.data[5,:,40,178], U[5,:,40,178]
#dis[sW], dis[sE], dis[sC]
#dis[[sW,sC,sE,259]]
#Q = u1[2,:,:] * areaW[:,:,xs].data
sW, sE, sC

(120, 235, 178)

In [4]:
Q1out = np.empty(len(time1))
#Q2out = np.empty(len(time))
Q1in = np.empty(len(time1))
#Q2in = np.empty(len(time))
for t in range(len(time1)):
    Q1 = u1[t,:,:] * areaW.data[:,:,xs]
    #Q2 = u2[t,:,:] * Area * HFacW
    Q1out[t] = Q1[Q1 > 0].sum()
    #Q2out[t] = Q2[Q2 > 0].sum()
    Q1in[t] = Q1[Q1 < 0].sum()
    #Q2in[t] = Q2[Q2 < 0].sum()

np.mean(Q1out[1]+Q1in[1])
#Q1in[[35,38]], Q1out[[35,38]]

245.10893477126956

In [5]:
Q1out, Q1in

(array([   0.        ,  245.10893477,  245.12898023,  245.00880468,
         231.49879301,  224.40016906,  277.67200203,  287.9825761 ,
         323.40478623,  362.58627636,  521.57910227,  846.84347858,
        1408.28357244, 2196.74435415, 3168.85298196, 4195.79293613,
        5152.23547244, 5933.88286046, 6500.26362433, 6885.80739462,
        7136.39848825, 7294.00933628, 7386.48366012, 7436.46738134,
        7468.76461432, 7484.29945584, 7486.03349554, 7474.70369105,
        7454.96503734, 7430.74034264, 7402.26944024, 7367.89310197,
        7328.54950229, 7290.80919684, 7259.19843416, 7231.18665022,
        7205.42670211, 7184.25069302, 7171.45162076, 7163.92058539,
        7160.16093981, 7156.17739565, 7158.14225725, 7164.25600543,
        7171.65363887, 7176.51929287, 7181.58070575, 7188.97513197,
        7197.48449356, 7204.39187512, 7209.50750024, 7213.17189527,
        7217.55965855, 7223.18377799, 7228.76276635, 7232.6023099 ,
        7235.82849879, 7240.84310245, 7246.41068

In [ ]:
#fig_path = "/Users/weiyangbao/Documents/Results_Figs/VolumeTest/"
#if not os.path.exists(fig_path):
#    os.makedirs(fig_path)

fig, (ax1, ax2, ax3) = plt.subplots(3,1, sharex=True, figsize=(12,12)) 
ax1.plot(time1,Q1in+Q1out, label='ConserveMass OFF')
#ax1.plot(time,Q2in+Q2out, label='ConserveMass ON')
ax1.set_title('$Q_{net}$', size=15, weight='bold')
#ax1.legend(loc = 'best', fontsize=15)
ax1.tick_params(labelsize=15)
ax1.xaxis.set_visible(False)
ax1.set_ylabel('Volume flux ($m^3/s$)', size=15)


ax2.plot(time1,Q1out, label='ConserveMass OFF')
#ax2.plot(time,Q2out, label='ConserveMass ON')
#ax2.axvline(x=275, color='b',lw=2)

ax2.set_title('$Q_1$', size=15, weight='bold')
#ax2.legend(loc = 'best', fontsize=15)
ax2.tick_params(labelsize=15)
ax2.xaxis.set_visible(False)
ax2.set_ylabel('Volume flux ($m^3/s$)', size=15)


ax3.plot(time1,Q1in, label='ConserveMass OFF')
#ax3.plot(time,Q2in, label='ConserveMass ON')
#ax3.axvline(x=275, color='b',lw=2)
ax3.set_title('$Q_2$', size=15, weight='bold')
#ax3.legend(loc = 'best', fontsize=15)
ax3.tick_params(labelsize=15)

ax3.set_xlabel('Time (d)', size=15)
ax3.set_ylabel('Volume flux ($m^3/s$)', size=15)
Q1in+Q1out

In [ ]:
N = np.arange(1,41,1)
N / 1.03505, time1[-1], time1[90]

In [ ]:
def volflux_ts(file, xidx):
    State01 = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/qf700_strat_high/state_' + str(format(file,'03d')) + '.nc')
    Grid01 = xr.open_dataset('/Volumes/Extreme SSD/MITgcm outputs/qf700_strat_high/grid_' + str(format(file,'03d')) + '.nc')
    T0 = 1 # Start time
    Tn = len(State01.T) # End time
    State = State01.isel(X=range(1,120), Xp1=range(1,121), Y=range(30,39), T=range(T0,Tn))
    Grid = Grid01.isel(X=range(1,120), Xp1=range(1,121), Y=range(30,39))    
    
    time = State.T.data / 86400
    
    Area = np.empty([50, 9]) # Cross Y direction
    Area[:10, :] = 100
    Area[10:20, :] = 200
    Area[20:30, :] = 300
    Area[30:40, :] = 800
    Area[40:, :] = 1600

    HFacW = Grid.HFacW.data[:,:,xidx] # vertical fraction of open cell at West face
    
    Q1 = np.empty(len(time))
    Q2 = np.empty(len(time))
    for t in range(len(time)):
        Q = State.U.data[t,:,:,xidx] * Area * HFacW
        Q1[t] = Q[Q > 0].sum()
        Q2[t] = Q[Q < 0].sum()
    
    return time, Q1, Q2

In [ ]:
prof = np.array([1,2,3,4])
hsr = np.array([9, 12, 15, 18]) / 150
xi = 47
fig1 = plt.figure(figsize=(10,10))
fig1.tight_layout()

ax1 = fig1.add_subplot(211)
#ax1.axvline(x=4.8, color='k',lw=1.5,ls='--')
ax1.set_xlim([0, 30])
ax1.set_title(r'(a) Down fjord', loc='center', size=15)
#ax1.xaxis.set_visible(False)
ax1.set_ylabel(r'Volume Flux [$m^3/s$]', size=15)
ax1.grid(True)
ax1.set_xticklabels([])
ax1.spines["top"].set_visible(False)
ax1.spines["right"].set_visible(False)



ax2 = fig1.add_subplot(212)
ax2.set_xlim([0, 30])
ax2.set_title(r'(b) Up fjord', loc='center', size=15)
ax2.set_ylabel(r'volume Flux [$m^3/s$]', size=15)
ax2.set_xlabel('Time [d]', size=15)
ax2.grid(True)
ax2.spines["top"].set_visible(False)
ax2.spines["right"].set_visible(False)

#ax3 = fig1.add_subplot(313)
#ax3.set_xlim([0, 30])
#ax3.set_title(r'(c) Net', loc='center', size=15)
#ax3.set_xlabel('Time [d]', size=15)
#ax3.set_ylabel('Heat Flux [J/s]', size=15)
#ax3.grid(True)
#ax3.spines["top"].set_visible(False)
#ax3.spines["right"].set_visible(False)

for i in range(len(prof)):
    ot, qp, qn = volflux_ts(prof[i],xi)
    
    ax1.plot(ot, qp, lw=2, label=str(prof[i]))
    ax1.legend(title=r'case', loc='right', fontsize=12)
    
    ax2.plot(ot, qn, lw=2, label=str(prof[i]))
    ax2.legend(title=r'case', loc='right', fontsize=12)
    
    #ax3.plot(ot, advhp+advhn, lw=2, label=str(hsr[i]))
    #ax3.legend(title=r'$h_s/h$', loc='right', fontsize=12)

In [ ]:
#fig_path = "/Users/weiyangbao/Documents/Results_Figs/VolumeTest/"

fig_path ="/home/1959/Jupyter_Figs/Qsg500n_004/"
if not os.path.exists(fig_path):
    os.makedirs(fig_path)

fig.savefig(fig_path + '/Q_test.png', dpi=200, bbox_inches='tight')    